In [ ]:
data = pd.read_excel("dataset.xlsx")                              # Load data (feature-target)
x = data.drop(columns=["Label"])                      
y = data["Label"]

x = RobustScaler().fit_transform(x)                               # Scaling and split data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y_res)

smote_enn = SMOTEENN(sampling_strategy='auto', random_state=42)   # Resampling 
x_train_r, y_train_r = smote_enn.fit_resample(x_train, y_train)

param_grid = {                                                    # Tuning GBC
    'n_estimators': [50, 100, 200],               
    'learning_rate': [0.01, 0.1, 0.2],            
    'max_depth': [3, 5, 7],                       
    'subsample': [0.8, 1.0]                       
}

gbc = GradientBoostingClassifier()
grid_search = GridSearchCV(estimator=gbc, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train_r, y_train_r)
best_gbc = grid_search.best_estimator_
best_gbc.fit(x_train_r, y_train_r)
best_gbc = grid_search.best_estimator_

feature_map = ZFeatureMap(feature_dimension=x_train_r.shape[1], reps=1)  # QSVC
quantum_kernel = FidelityQuantumKernel(feature_map=feature_map)
qsvc = QSVC(quantum_kernel=quantum_kernel)
qsvc.fit(x_train_r, y_train_r)

y_pred_tr_gbc = best_gbc.predict(x_train_r).reshape(-1, 1)               # Evaluation
y_pred_ts_gbc = best_gbc.predict(x_test).reshape(-1, 1)
y_pred_tr_qsvc = qsvc.predict(x_train_r)
y_pred_ts_qsvc = qsvc.predict(x_test)
print(classification_report(y_train_r, y_pred_tr_gbc))
print(classification_report(y_test, y_pred_ts_gbc))
print(classification_report(y_train_r, y_pred_tr_qsvc))
print(classification_report(y_test, y_pred_ts_qsvc))